# 1. 取得資訊

In [ ]:
import boto3
import json
from PIL import Image
import base64
from io import BytesIO

# 設定你的 Endpoint 名稱
endpoint_name = ""

# 建立 SageMaker 的 client
sm_client = boto3.client('sagemaker')

try:
    # 呼叫 API 查詢你的 Endpoint
    response = sm_client.list_inference_components(
        EndpointNameEquals=endpoint_name
    )

    # 取得 inference compenent name
    components = response.get('InferenceComponents', [])
    inference_component = components[0]['InferenceComponentName']

    # 顯示資訊
    print("查詢成功！以下是 Endpoint 以及 Component 的資訊：")
    print('='*50)
    for info in components[0]:
        print(f'{info}: {components[0][info]}')

except Exception as e:
    print(f"查詢失敗，錯誤訊息：{e}")

# 2. 發送請求

In [ ]:
# 建構 boto3 client
runtime = boto3.client('runtime.sagemaker')

# 準備 prompt
prompt = ""

try:
    # 發送請求
    response = runtime.invoke_endpoint(
        EndpointName=endpoint_name,
        InferenceComponentName=inference_component,
        ContentType='application/json',
        Accept='application/json;jpeg',
        Body=json.dumps({
            "prompt": prompt,
            "guidance_scale": 7.5,
            "num_inference_steps": 30,
        })
    )

    # 解析回傳的圖片
    response_body = json.loads(response['Body'].read().decode())
    print(f"生成完畢！\n回傳格式：{response_body.keys()}")

except Exception as e:
    print(f"生成失敗，錯誤訊息：{e}")

# 3. 顯示圖片

In [ ]:
try:
    image_data = response_body['generated_images'][0]
    image_bytes = base64.b64decode(image_data)
    image = Image.open(BytesIO(image_bytes))
    display(image)

except Exception as e:
    print(f"Base64 解碼失敗，錯誤訊息：{e}")